# 🎨 Data Designer 101: The Basics

[Click here](https://raw.githubusercontent.com/NVIDIA-NeMo/DataDesigner/refs/heads/main/docs/notebooks/1-the-basics.ipynb) to download this notebook to your computer.

#### 📚 What you'll learn

This notebook demonstrates the basics of Data Designer by generating a simple product review dataset.


### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    CategorySamplerParams,
    DataDesigner,
    DataDesignerConfigBuilder,
    InferenceParameters,
    LLMTextColumnConfig,
    ModelConfig,
    PersonFromFakerSamplerParams,
    SamplerColumnConfig,
    SamplerType,
    SubcategorySamplerParams,
    UniformSamplerParams,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object is responsible for managing the data generation process.

- When initialized without arguments, the [default model providers](https://nvidia-nemo.github.io/DataDesigner/models/default-model-settings/) are used.


In [2]:
data_designer = DataDesigner()

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](https://nvidia-nemo.github.io/DataDesigner/models/default-model-settings/) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nvidia-nemotron-nano-9b-v2"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v2"

# This sets reasoning to False for the nemotron-nano-v2 model.
SYSTEM_PROMPT = "/no_think"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.5,
            top_p=1.0,
            max_tokens=1024,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🎲 Getting started with sampler columns

- Sampler columns offer non-LLM based generation of synthetic data.

- They are particularly useful for **steering the diversity** of the generated data, as we demonstrate below.

<br>

You can view available samplers using the config builder's `info` property:


In [5]:
config_builder.info.display("samplers")

─────────────────────────────────────────── NeMo Data Designer Samplers ───────────────────────────────────────────

┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Type               ┃ Parameter                ┃ Data Type                         ┃ Required ┃ Constraints      ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ bernoulli          │ p                        │ number                            │    ✓     │ >= 0.0, <= 1.0   │
│                    │ sampler_type             │ string                            │          │                  │
├────────────────────┼──────────────────────────┼───────────────────────────────────┼──────────┼──────────────────┤
│ bernoulli_mixture  │ p                        │ number                            │    ✓     │ >= 0.0, <= 1.0   │
│                    │ dist_name                │ string                            │    ✓     │                  │
│                    │ dist_params              │ dict                              │    ✓     │                  │
│                    │ sampler_type             │ string                            │          │                  │
├────────────────────┼──────────────────────────┼───────────────────────────────────┼──────────┼──────────────────┤
│ binomial           │ n                        │ integer                           │    ✓     │                  │
│                    │ p                        │ number                            │    ✓     │ >= 0.0, <= 1.0   │
│                    │ sampler_type             │ string                            │          │                  │
├────────────────────┼──────────────────────────┼───────────────────────────────────┼──────────┼──────────────────┤
│ category           │ values                   │ string[] | integer[] | number[]   │    ✓     │ len > 1          │
│                    │ weights                  │ number[] | null                   │          │                  │
│                    │ sampler_type             │ string                            │          │                  │
├────────────────────┼──────────────────────────┼───────────────────────────────────┼──────────┼──────────────────┤
│ datetime           │ start                    │ string                            │    ✓     │                  │
│                    │ end                      │ string                            │    ✓     │                  │
│                    │ unit                     │ string                            │          │                  │
│                    │ sampler_type             │ string                            │          │                  │
├────────────────────┼──────────────────────────┼───────────────────────────────────┼──────────┼──────────────────┤
│ gaussian           │ mean                     │ number                            │    ✓     │                  │
│                    │ stddev                   │ number                            │    ✓     │                  │
│                    │ decimal_places           │ integer | null                    │          │                  │
│                    │ sampler_type             │ string                            │          │                  │
├────────────────────┼──────────────────────────┼───────────────────────────────────┼──────────┼──────────────────┤
│ person             │ locale                   │ string                            │          │                  │
│                    │ sex                      │ string | null                     │          │                  │
│                    │ city                     │ string | string[] | null          │          │                  │
│                    │ age_range                │ integer[]                         │          │ len > 2, len < 2 │
│                    │ select_field_values      │ objec

Let's start designing our product review dataset by adding product category and subcategory columns.


In [6]:
config_builder.add_column(
    SamplerColumnConfig(
        name="product_category",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "Electronics",
                "Clothing",
                "Home & Kitchen",
                "Books",
                "Home Office",
            ],
        ),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="product_subcategory",
        sampler_type=SamplerType.SUBCATEGORY,
        params=SubcategorySamplerParams(
            category="product_category",
            values={
                "Electronics": [
                    "Smartphones",
                    "Laptops",
                    "Headphones",
                    "Cameras",
                    "Accessories",
                ],
                "Clothing": [
                    "Men's Clothing",
                    "Women's Clothing",
                    "Winter Coats",
                    "Activewear",
                    "Accessories",
                ],
                "Home & Kitchen": [
                    "Appliances",
                    "Cookware",
                    "Furniture",
                    "Decor",
                    "Organization",
                ],
                "Books": [
                    "Fiction",
                    "Non-Fiction",
                    "Self-Help",
                    "Textbooks",
                    "Classics",
                ],
                "Home Office": [
                    "Desks",
                    "Chairs",
                    "Storage",
                    "Office Supplies",
                    "Lighting",
                ],
            },
        ),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="target_age_range",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(values=["18-25", "25-35", "35-50", "50-65", "65+"]),
    )
)

# Optionally validate that the columns are configured correctly.
config_builder.validate()

[10:16:58] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    sampler_columns: ['product_category', 'product_subcategory', 'target_age_range']
)

Next, let's add samplers to generate data related to the customer and their review.


In [7]:
config_builder.add_column(
    SamplerColumnConfig(
        name="customer",
        sampler_type=SamplerType.PERSON_FROM_FAKER,
        params=PersonFromFakerSamplerParams(age_range=[18, 70], locale="en_US"),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="number_of_stars",
        sampler_type=SamplerType.UNIFORM,
        params=UniformSamplerParams(low=1, high=5),
        convert_to="int",  # Convert the sampled float to an integer.
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="review_style",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["rambling", "brief", "detailed", "structured with bullet points"],
            weights=[1, 2, 2, 1],
        ),
    )
)

config_builder.validate()

[10:16:58] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    sampler_columns: [
        "product_category",
        "product_subcategory",
        "target_age_range",
        "customer",
        "number_of_stars",
        "review_style"
    ]
)

## 🦜 LLM-generated columns

- The real power of Data Designer comes from leveraging LLMs to generate text, code, and structured data.

- When prompting the LLM, we can use Jinja templating to reference other columns in the dataset.

- As we see below, nested json fields can be accessed using dot notation.


In [8]:
config_builder.add_column(
    LLMTextColumnConfig(
        name="product_name",
        prompt=(
            "You are a helpful assistant that generates product names. DO NOT add quotes around the product name.\n\n"
            "Come up with a creative product name for a product in the '{{ product_category }}' category, focusing "
            "on products related to '{{ product_subcategory }}'. The target age range of the ideal customer is "
            "{{ target_age_range }} years old. Respond with only the product name, no other text."
        ),
        system_prompt=SYSTEM_PROMPT,
        model_alias=MODEL_ALIAS,
    )
)

config_builder.add_column(
    LLMTextColumnConfig(
        name="customer_review",
        prompt=(
            "You are a customer named {{ customer.first_name }} from {{ customer.city }}, {{ customer.state }}. "
            "You are {{ customer.age }} years old and recently purchased a product called {{ product_name }}. "
            "Write a review of this product, which you gave a rating of {{ number_of_stars }} stars. "
            "The style of the review should be '{{ review_style }}'."
        ),
        system_prompt=SYSTEM_PROMPT,
        model_alias=MODEL_ALIAS,
    )
)

config_builder.validate()

[10:16:58] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    sampler_columns: [
        "product_category",
        "product_subcategory",
        "target_age_range",
        "customer",
        "number_of_stars",
        "review_style"
    ]
    llm_text_columns: ['product_name', 'customer_review']
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [9]:
preview = data_designer.preview(config_builder)

[10:16:58] [INFO] 🔭 Preview generation in progress


[10:16:58] [INFO] ✅ Validation passed


[10:16:58] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[10:16:58] [INFO] 🩺 Running health checks for models...


[10:16:58] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...


...

[10:17:16] [INFO]   |-- 🎲 column: 'number_of_stars'


[10:17:16] [INFO]   |-- 🎲 column: 'review_style'


[10:17:16] [INFO]   |-- 📝 column: 'product_name'


[10:17:16] [INFO]   |-- 📝 column: 'customer_review'


[10:17:16] [INFO] 🎉 Preview complete!


In [10]:
# Run this cell multiple times to cycle through the 10 preview records.
preview.display_sample_record()

                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                ┃ Value                                                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_category    │ Home Office                                                                               │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ product_subcategory │ Lighting                                                                                  │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ target_age_range    │ 25-35                                                                                     │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ customer            │ {                                                                                         │
│                     │     'uuid': '6d8d6665-1cf5-4a3b-8a61-bf535a0f3f45',                                       │
│                     │     'locale': 'en_US',                                                                    │
│                     │     'first_name': 'Rachel',                                                               │
│                     │     'last_name': 'Gomez',                                                                 │
│                     │     'middle_name': None,                                                                  │
│                     │     'sex': 'Female',                                                                      │
│                     │     'street_number': '5879',                                                              │
│                     │     'street_name': 'David Manors',                                                        │
│                     │     'city': 'South Savannah',                                                             │
│                     │     'state': 'Mississippi',                                                               │
│                     │     'postcode': '32421',                                                                  │
│                     │     'age': 24,                                                                            │
│                     │     'birth_date': '2001-01-13',                                                           │
│                     │     'country': 'Thailand',                                                                │
│                     │     'marital_status': 'never_married',                                                    │
│                     │     'education_level': 'graduate',                                                        │
│                     │     'unit': '',                                                                           │
│                     │     'occupation': 'Exhibition designer',                                                  │
│                     │     'phone_number': '+1-521-347-6306x89692',                                              │
│                     │     'bachelors_field': 'arts_humanities'                                                  │
│                     │ }                                                                                         │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ number_of_stars     │ 4                                                                                         │
├─────────────────────┼─────────────────────────────────

In [11]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,product_category,product_subcategory,target_age_range,customer,number_of_stars,review_style,product_name,customer_review
0,Home Office,Lighting,25-35,{'uuid': '6d8d6665-1cf5-4a3b-8a61-bf535a0f3f45...,4,brief,LumenNest\n,**LumenNest Review (4/5):** \nGreat product o...
1,Home Office,Chairs,65+,{'uuid': '4397a179-1a8d-4e4f-93cc-3e6658249e87...,4,rambling,ErgoEasy Chair\n,"Well, hello there! So, I’m Stephanie, from Nor..."
2,Home & Kitchen,Furniture,25-35,{'uuid': 'a731bf79-f464-45ad-98b7-02f0bafbacf6...,2,rambling,UrbanNest\n,"Okay, so I bought this thing called UrbanNest,..."
3,Books,Fiction,18-25,{'uuid': '2f1f70c1-08ee-4999-9a15-000892f0390b...,1,brief,Whispers of the Unseen\n,**Review:** \nAvoid. Disappointing quality an...
4,Electronics,Cameras,35-50,{'uuid': '27bc4314-c2b5-4bfd-bac2-d2dec5fc8c85...,2,brief,LensCraft Pro\n,**Review:** LensCraft Pro is overpriced and un...
5,Electronics,Accessories,18-25,{'uuid': '498a6772-298c-4351-87b6-c410504396be...,2,structured with bullet points,NeonNest\n,**Review of NeonNest by Jeffery from East Donn...
6,Electronics,Headphones,50-65,{'uuid': '91384595-979f-4dc5-b46b-8206701e1150...,3,brief,SereneSound Headphones\n,**Crystal's Review (3 stars):** \nDecent soun...
7,Home & Kitchen,Appliances,35-50,{'uuid': 'a66658b8-2cb2-41c9-b2ad-2318c86e1475...,3,brief,SmartChef Pro\n,**Review:** \nSmartChef Pro is decent but not...
8,Home Office,Lighting,50-65,{'uuid': '599d610b-8dba-49a3-a316-28219ac94640...,2,rambling,WarmGlow Desk Lamp\n,"Well, I guess I should start by saying that I ..."
9,Books,Self-Help,35-50,{'uuid': '5d80443a-df94-4265-a69f-51616fc83bd4...,4,structured with bullet points,MindfulMoments\n,"**MindfulMoments Review (4 Stars) – Megan, New..."


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [12]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 8                               │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                    ┃       data type ┃            number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_category               │          string │                       4 (40.0%) │                   category │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ product_subcategory            │          string │                       9 (90.0%) │                subcategory │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ target_age_range               │          string │                       5 (50.0%) │                   category │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ customer                       │            dict │                     10 (100.0%) │          person_from_faker │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ number_of_stars                │             int │                       4 (40.0%) │                    uniform │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ review_style                   │          string │                       3 (30.0%) │                   category │
└────────────────────────────────┴─────────────────┴─────────────────────────────────┴────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_name          │        string │                10 (100.0%) │      77.0 +/- 0.8 │            5.0 +/- 1.0 │
├───────────────────────┼───────────────┼─────────────────

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [13]:
results = data_designer.create(config_builder, num_records=20)

[10:17:16] [INFO] 🎨 Creating Data Designer dataset


[10:17:16] [INFO] ✅ Validation passed


[10:17:16] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[10:17:16] [INFO] 🩺 Running health checks for models...


[10:17:16] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...


...

[10:18:07] [INFO]   |-- 🎲 column: 'customer'


[10:18:07] [INFO]   |-- 🎲 column: 'number_of_stars'


[10:18:07] [INFO]   |-- 🎲 column: 'review_style'


[10:18:07] [INFO]   |-- 📝 column: 'product_name'


[10:18:07] [INFO]   |-- 📝 column: 'customer_review'


In [14]:
# Load the generated dataset as a pandas DataFrame.
dataset = results.load_dataset()

dataset.head()

,product_category,product_subcategory,target_age_range,customer,number_of_stars,review_style,product_name,customer_review
0,Home Office,Storage,65+,"{'age': 22, 'bachelors_field': 'no_degree', 'b...",4,brief,SeniorSafeStorage,**Review:** Great product for seniors! Easy ...
1,Books,Fiction,65+,"{'age': 55, 'bachelors_field': 'stem_related',...",4,detailed,Whispers of Yesterday,**Title: A Sentimental Journey Through Time – ...
2,Electronics,Laptops,65+,"{'age': 33, 'bachelors_field': 'business', 'bi...",2,detailed,GrandPad Laptop,**Product Review: GrandPad Laptop (2 Stars)** ...
3,Clothing,Women's Clothing,50-65,"{'age': 24, 'bachelors_field': 'no_degree', 'b...",2,rambling,EleganceEssentials,"Okay, so I need to write a rambling review fro..."
4,Clothing,Men's Clothing,35-50,"{'age': 44, 'bachelors_field': 'no_degree', 'b...",3,detailed,Timeless Edge,**Product Review: Timeless Edge – A Detailed L...


In [15]:
# Load the analysis results into memory.
analysis = results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 20                              │ 8                               │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                    ┃       data type ┃            number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_category               │          string │                       5 (25.0%) │                   category │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ product_subcategory            │          string │                      11 (55.0%) │                subcategory │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ target_age_range               │          string │                       5 (25.0%) │                   category │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ customer                       │            dict │                     20 (100.0%) │          person_from_faker │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ number_of_stars                │             int │                       5 (25.0%) │                    uniform │
├────────────────────────────────┼─────────────────┼─────────────────────────────────┼────────────────────────────┤
│ review_style                   │          string │                       3 (15.0%) │                   category │
└────────────────────────────────┴─────────────────┴─────────────────────────────────┴────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_name          │        string │                20 (100.0%) │      77.0 +/- 1.1 │            4.0 +/- 0.8 │
├───────────────────────┼───────────────┼─────────────────

## ⏭️ Next Steps

Now that you've seen the basics of Data Designer, check out the following notebooks to learn more about:

- [Structured outputs and jinja expressions](/notebooks/2-structured-outputs-and-jinja-expressions/)

- [Seeding synthetic data generation with an external dataset](/notebooks/3-seeding-with-a-dataset/)
